# Gestion des erreurs, assertion et alertes
----

## Codes d'erreurs

Lorsque python doit exécuter une instruction non valide (c'est à dire une instruction qui bug ou qui n'a pas de sens), il arrête l'exécution et affiche un message d'erreur. Ce message est indispensable pour pouvoir corriger l'erreur. Par exemple, lorsque vous faite `1/0`, voici ce qui s'affiche (les premières lignes du message peuvent varier entre python et les notebook mais la dernière ligne sera toujours identique) :

In [2]:
1/0

ZeroDivisionError: division by zero

In [3]:
non_def

NameError: name 'non_def' is not defined

In [4]:
"a"/2

TypeError: unsupported operand type(s) for /: 'str' and 'int'

Python indique explicitement que l'erreur vient de la division par zéro. Ici, c'est évident mais lorsque vous coder une fonction (de plus d'une ligne) qui plante, ces messages sont indispensables pour pouvoir corriger votre erreur.

*Remarque :
 	Si le message d'erreur qui s'affiche est long, partez du bas (si vous appelez une fonction qui appelle un fonction qui elle-même comporte une erreur, python affichera ces trois appels de fonctions !). De même, il arrive parfois que l'erreur soit faite à ligne précédent celle afficher par l'interpréteur python (par exemple lorsque vous oubliez de fermer la parenthèse à la ligne du dessus ou pour certaines erreurs d'indentation).*

### Erreurs classiques
 + division by zero
 + not callable` : seules les fonctions peuvent être appelée. Par ex A(2) au lieu de A[2] pour A tableau
 + not subscriptable : erreur inverse, mauvaise utilisation des crochets.
 + out of bounds : accès à un éléments en dehors de la liste, du tableau, ...
 + too many indices : trop d'indice pour un tableau (ex A[1,1] pour un tableau 1D)



### Définir ces propres erreurs

Lorsque que vous créer vos fonctions de nouvelles erreurs ou comportement anormaux, non prévus par python, peuvent apparaître. Vous pouvez alors définir vos propres message d'erreur.

En voici un exemple : on écrit un code qui converti un temps XhYm en secondes.

In [1]:
def convert_time():
  a = input("Entrer une duree au format XXhYYm : ")
  i = a.find('h') # Renvoie la position de 'h' dans la chaine de caracteres
  h = int(a[:i])    # Heures
  m = int(a[i+1:])  # Minutes
  #print(i,h,m)
  print("Cela représente ",3600*h+60*m, " secondes")

Si l'utilisateur saisi "10m" au lieu de saisir "0h10m", la première fonction ne plante pas mais affiche un résultat faux (4200 secondes) ; la seconde fonction détecte le problème (mauvaise saisie) et s'arrête. En effet, la commande `find` renvoie "-1" quand elle ne trouve pas la chaine de caractère trouvée.

In [2]:
convert_time()

Entrer une duree au format XXhYYm :  6m


ValueError: invalid literal for int() with base 10: '6m'

In [5]:
def convert_time_safe():
  a = input("Entrer une duree au format XXhYYm : ")
  i = a.find('h')
  print(i)
  if i<0 : 
        raise ValueError("La saisie ne contient pas le caratère 'h'")
  h = int(a[:i])    # Minutes
  m = int(a[i+1:])  # Minutes
  print("Cela représente ",3600*h+60*m, " secondes")

In [4]:
convert_time_safe()

Entrer une duree au format XXhYYm :  -1


-1


ValueError: La saisie ne contient pas le caratère 'h'

**La synthaxe pour renvoyer une erreur définie par l'utilisateur est :**

In [6]:
raise ValueError("Message d'erreur définit par l'utilisateur.")

ValueError: Message d'erreur définit par l'utilisateur.

----
# Intercepter les erreurs

Lorsque python rencontre une erreur il arrête le programme de façon brutale. Il existe des cas où l'on voudrait soit poursuivre l'éxecution ou encore arrêter le programme après une sauvegarde ou un arrêt plus doux.

Il est alors possible d'intercepter l'erreur pour la gérer de la façon de notre choix. C'est en particulier possible pour les valeur saisies par l'utilisateur qui peuvent être à la source de nombreux bugs.

Dans ces cas là, on peut par exemple utiliser une valeur par défaut ou demander à l'utilisateur de ressaisir une nouvelle valeur :

In [9]:
def f0():
  b = input("Entrer un nombre :")
  # Instruction qui peut générer une erreur
  a = int(b)
  return a

f0()

Entrer un nombre : toto


ValueError: invalid literal for int() with base 10: 'toto'

In [12]:
def f0():
  b = input("Entrer un nombre :")
  try:
    # Instruction qui peut générer une erreur
    a = int(b)
  except ValueError:
    # Instructions à faire en cas d'erreur
    print("vous n'avez pas saisi un nombre, on remplace par 0")
    a = 0
  return a

f0()

Entrer un nombre : toto


vous n'avez pas saisi un nombre, on remplace par 0


0

Le code essaye de faire l'instruction du try ; si elle n'y parvient pas, elle exécute les commandes du bloc except.
Il est aussi possible d'ajouter un bloc à n'exécuter que si tout ces bien passer ; ce bloc est introduit par finally :

In [15]:
def f1():
    b = input("Entrer un nombre :")
    try:
        # Instruction qui peut générer une erreur
        a = int(b)
    except ValueError:
        # Instructions à faire en cas d'erreur
        print("vous n'avez pas saisi un nombre, on remplace par 0")
        a = 0
    else :
        print("Vous avez bien saisi un nombre !")
    return a

f1()

Entrer un nombre : toto


vous n'avez pas saisi un nombre, on remplace par 0


0

Cette synthaxe est aussi souvent utilisée pour la manipulation de fichiers ou les source d'erreurs peuvent être fréquente :

In [18]:
fname=input("Entrer le nom d'un fichier")

try :
    f=open(fname,'r')
except FileNotFoundError: 
    print(f"{fname} n'a pas été trouvé")
except IsADirectoryError:
    print(f"{fname} est un dossier !")
except ValueError:
    # all other types of exceptions
    print("Il y a eu une erreur !")
else :
    # Seulement si on a pu ouvrir correctement le fichier
    contenu=f.read()    
    print(contenu)
finally :
    f.close() 
    # On s'assure de fermer dans tous les cas le fichier quoiqu'il arrive.

Entrer le nom d'un fichier hello.txt


hello. Je suis un fichier.


----
## Alertes

Python permet de définir explicitement des messages comme étant important mais ne nécessitant pas l'arrêt du programme. On les appelles alertes.

In [19]:
import warnings as w

def f3():
  b = input("Entrer un nombre :")
  try:
    a = int(b)
  except ValueError:
    w.warn("Pas un nombre, on remplace par 0")
    a = 0
  return a

In [8]:
f3()

Entrer un nombre : toto


<ipython-input-7-70ddbbda7c2b>:8: UserWarning: Pas un nombre, on remplace par 0
  w.warn("Pas un nombre, on remplace par 0")


0

----
## Assertions

Cette fonction ont besoin de conditions sur leurs entrées pour bien fonctionner. Les assertions permette de de vérifier que ces conditions sont bien vérifiées.

*Remarque : il est aussi possible de faire des tester et de renvoyer des erreurs (raise ValueError) mais les assertions permettent de faire cela plus rapidement pour des conditions simples.*


In [29]:
def KelvinToFahrenheit(Temperature):
    """Convertion d'une température en degrès kelvin vers des Fahrendheits."""
    assert (Temperature >= 0),"Colder than absolute zero!"
    return ((Temperature-273)*1.8)+32


In [30]:
print(KelvinToFahrenheit(273))
print(int(KelvinToFahrenheit(505.78)))
print(KelvinToFahrenheit(-5))

32.0
451


AssertionError: Colder than absolute zero!

*Remarque :*
    Ce code aurai pu être écrit avec :
```
if (Temperature <0):
    raise ValueError("Colder than absolute zero")
```

*(hors programme)
Mais il est possible de désactiver d'un coup tous les assertions d'un code (en quittant le mode dit `debug` actif par défaut) alors qu'il n'est jamais possible de désactiver la levée d'une erreur.
L'idée sous jacente est de tester le code avec les assertions, qui sont très utiles pour débugger un code mais qui coute (un peu) de temps de calcul puis de les désactiver une fois le code valider pour obtenir un code (un peu) plus rapide.
On garde alors la levée d'erreur classique pour des situations inattendue ou résultant d'un mauvaise usage de l'utilisateur.*

In [33]:
a = 1

In [34]:
b = a

In [35]:
b= 2

In [36]:
print(a)

1
